## Data preprocessing for Women in Olympics visualization

This notebook implements the required data preprocessing to create the dataset used for the visualization project "Women in Olympics visualization. <br>
In the notebook it is merged two dataset with the athletes that have participated in the different Olympic games, and it also integrates some geographical information with the continent of each country and the parameter called HDI representing the Human Development Index of the different countries. All this information will be used in the data visualization published in Tableau Desktop. <br>

## 1.- Load and process dataset with Athletes in games up to Rio 2016

In [1]:
# import packages
import pandas as pd
import numpy as np

In [2]:
# NOC names normalization
NOC_converter = dict(ANZ='AUS',
                    ART='IOA',
                    ROT='IOA',
                    UNK='IOA',
                    BOH='CZE',
                    TCH='CZE',
                    CRT='GRE',
                    GDR='GER',
                    SAA='GER',
                    NFL='CAN',
                    URS='RUS',
                    ROC='RUS',
                    OAR='RUS',
                    EUN='RUS',
                    YAR='YEM',
                    YMD='YEM',
                    YUG='SRB',
                    SCG='SRB')

In [3]:
# load dataset
df = pd.read_csv(r'..\datasets\athlete_events.csv')

In [4]:
# add dummy columns if the athlete is Male or Female
df = df.join(pd.get_dummies(df['Sex']).astype(int))

In [5]:
# Normalizes NOC names for the same geographical areas at different points of the history
NOC_names = df[['NOC', 'Team', 'Year']].value_counts()
NOC_names = NOC_names.reset_index().sort_values(by=['Year', 'count'], ascending=False)

NOC_names_dict = dict()
for noc in NOC_names.itertuples():
    if noc.NOC not in NOC_names_dict.keys():
        NOC_names_dict[noc.NOC] = noc.Team
        
df['NOC'] = df['NOC'].replace(NOC_converter)

# set city to Melbourne for Olympic games in 1956 (equestrian events exception)
df.loc[df['Games'] == "1956 Summer", 'City'] = 'Melbourne' 

In [6]:
# pivot dataset by Season, Year, City and NOC
table = df.pivot_table(index=['Season', 'Year', 'City', 'NOC'], aggfunc={'M':'sum', 'F': 'sum'})
table['Total Athletes'] = table['F'] + table['M']
table['Gender equity'] = table['F'] / table['Total Athletes']
table

F    M  Total Athletes  Gender equity
Season Year City   NOC                                         
Summer 1896 Athina AUS    0    5               5       0.000000
                   AUT    0    8               8       0.000000
                   DEN    0   15              15       0.000000
                   FRA    0   26              26       0.000000
                   GBR    0   25              25       0.000000
...                     ...  ...             ...            ...
Winter 2014 Sochi  UKR   56   45             101       0.554455
                   USA  158  209             367       0.430518
                   UZB    2    3               5       0.400000
                   VEN    0    1               1       0.000000
                   ZIM    0    2               2       0.000000

[3832 rows x 4 columns]


## 2.- Load and process dataset with Athletes in games after Rio 2016

In [7]:
df_new = pd.read_csv(r'..\datasets\athletes_2018_to_2022.csv')

def get_sex(x):
    
    if x[-1] == 'M':
        out = 'M'
    elif x[-1] == 'W':
        out = 'F'
    else:
        out = 'Mixed'
    return out
df_new['Sex'] = df_new['Event'].apply(get_sex)
# add column if the athlete is Male or Female
df_new = df_new.join(pd.get_dummies(df_new['Sex']).astype(int))
df_new['NOC'] = df_new['NOC'].replace(NOC_converter)
df_new

,Season,City,Year,Sport,Event,Name,NOC,Sex,F,M,Mixed
0,Winter,PyeongChang,2018,Alpine Skiing,Alpine combined W,Michelle Gisin,SUI,F,1,0,0
1,Winter,PyeongChang,2018,Alpine Skiing,Alpine combined W,Mikaela Shiffrin,USA,F,1,0,0
2,Winter,PyeongChang,2018,Alpine Skiing,Alpine combined W,Wendy Holdener,SUI,F,1,0,0
3,Winter,PyeongChang,2018,Alpine Skiing,Alpine combined W,Ragnhild Mowinckel,NOR,F,1,0,0
4,Winter,PyeongChang,2018,Alpine Skiing,Alpine combined W,Petra Vlhova,SVK,F,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
24116,Winter,Beijing,2022,Speed Skating,Team pursuit M,Andrea Giovannini,ITA,M,0,1,0
24117,Winter,Beijing,2022,Speed Skating,Team pursuit M,Michele Malfatti,ITA,M,0,1,0
24118,Winter,Beijing,2022,Speed Skating,Team pursuit M,Lian Ziwen,CHN,M,0,1,0
24119,Winter,Beijing,2022,Speed Skating,Team pursuit M,Wang Haotian,CHN,M,0,1,0


In [8]:
# pivot dataset by Season, Year, City and NOC
table_new = df_new.pivot_table(index=['Season', 'Year', 'City', 'NOC'], aggfunc={'M':'sum', 'F': 'sum', 'Mixed':'sum'})
table_new

F    M  Mixed
Season Year City    NOC                 
Summer 2020 Tokyo   AFG    1    4      0
                    AHO    0    1      0
                    ALB    4    6      0
                    ALG   15   29      0
                    AND    2    1      0
...                      ...  ...    ...
Winter 2022 Beijing TTO    0    2      0
                    TUR    7    7      0
                    UKR   44   46      0
                    USA  162  175     24
                    UZB    0    2      0

[390 rows x 3 columns]

In [9]:
# split mixed events by 50% F and 50% M
table_new['F_in_mixed'] = table_new['Mixed'] // 2
table_new['M_in_mixed'] = table_new['Mixed'] - table_new['F_in_mixed']
table_new['F'] = table_new['F'] + table_new['F_in_mixed']
table_new['M'] = table_new['M'] + table_new['M_in_mixed']
table_new['Total Athletes'] = table_new['F'] + table_new['M']
table_new['Gender equity'] = table_new['F'] / table_new['Total Athletes']

In [10]:
# merge both datasets
df_all_games = pd.concat([table, table_new[table.columns]])

In [11]:
# reset index and includes Team name for each NOC, and last games mark
df_all_games = df_all_games.reset_index()
df_all_games['Team'] = df_all_games['NOC'].map(NOC_names_dict)
df_all_games['last_games'] = df_all_games['Year'].isin([2020, 2022])
df_all_games

,Season,Year,City,NOC,F,M,Total Athletes,Gender equity,Team,last_games
0,Summer,1896,Athina,AUS,0,5,5,0.000000,Australia,False
1,Summer,1896,Athina,AUT,0,8,8,0.000000,Austria,False
2,Summer,1896,Athina,DEN,0,15,15,0.000000,Denmark,False
3,Summer,1896,Athina,FRA,0,26,26,0.000000,France,False
4,Summer,1896,Athina,GBR,0,25,25,0.000000,Great Britain,False
...,...,...,...,...,...,...,...,...,...,...
4217,Winter,2022,Beijing,TTO,0,2,2,0.000000,Trinidad and Tobago,True
4218,Winter,2022,Beijing,TUR,7,7,14,0.500000,Turkey,True
4219,Winter,2022,Beijing,UKR,44,46,90,0.488889,Ukraine,True
4220,Winter,2022,Beijing,USA,174,187,361,0.481994,United States,True


In [12]:
# create structure with NOC and Team information
noc_names = df_all_games[['NOC', 'Team']].value_counts().index

## 3.- Load geographical information and integration of HDI 

In [13]:
# load list of countries
df_countries = pd.read_csv(r'..\datasets\List_of_countries_by_continent.txt', sep='\t')

In [14]:
# load HDI information
df_hdi = pd.read_csv(r'..\datasets\List_of_countries_by_HDI.txt', sep='\t')

In [15]:
# names converter for specific countries where name in datasets doesn't match
countries_hdi_converter = {'South Korea': 'Republic of Korea',
                          'Czech Republic': 'Czechia',
                          'Vietnam': 'Viet Nam',
                          'Cape Verde': 'Cabo Verde',
                          'Laos': "Lao People's Democratic Republic",
                          'São Tomé and Príncipe': 'Sao Tome and Principe',
                          'Ivory Coast': 'Côte d’Ivoire',
                          'DR Congo': 'Democratic Republic of the Congo'}

In [16]:
# add HDI to list of countries
df_countries['HDI'] = np.nan

for c, hdi in zip(df_hdi['Country or territory'], df_hdi['HDI value']):
    c = c.strip()
    if c in countries_hdi_converter.keys():
        c = countries_hdi_converter[c]
        
    f = df_countries['Country or Area'].str.contains(c, case=False, regex=False)
    if any(f):
        df_countries.loc[f, 'HDI'] = hdi
   
    else:
        print('-{}-'.format(c))

In [17]:
# assign average value of HDIin the subregion for the missed countries
f_no_nan = ~np.isnan(df_countries['HDI'])
hdi_table = df_countries[f_no_nan].pivot_table(index='Region 1', aggfunc={'HDI':'mean'})

df_countries.loc[~f_no_nan, 'HDI'] = df_countries.loc[~f_no_nan, 'Region 1'].map(hdi_table.to_dict()['HDI'])


## 3.1 Region and continent integration in all games athletes dataset

In [18]:
# names converter for specific countries where name in datasets doesn't match
countries_converters = {'Chinese Taipei': ('Asia', 'Eastern Asia'),
                        'North Korea': ('Asia', 'Eastern Asia'),
                        'Congo (Brazzaville)': ('Africa', 'Middle Africa'),
                        'Swaziland': ('Africa', 'Southern Africa'),
                        'Congo (Kinshasa)': ('Africa', 'Middle Africa'),
                        'Guinea Bissau': ('Africa', 'Western Africa'),
                        'North Borneo': ('Asia', 'South-eastern Asia'),
                         'United Arab Republic': ('Asia', 'Western Asia'),
                        'West Indies Federation': ('North America', 'Caribbean'),
                        'Rhodesia': ('Africa', 'Eastern Africa'),
                       'Netherlands Antilles':('North America', 'Caribbean'),
                       'West Germany': ('Europe', 'Western Europe'),
                       'Vietnam': ('Asia', 'South-eastern Asia'),
                       'Kosovo': ('Europe', 'Southern Europe'),
                        'Malaya': ('Asia', 'South-eastern Asia'),
                       'Individual Olympic Athletes': ('Individual Olympic Athletes', 'Individual Olympic Athletes')}

In [19]:
# integrates Region, Continent and HDI information

df_all_games['Region'] = ''
df_all_games['Continent'] = ''
df_all_games['HDI'] = np.nan
team_not_found = []

for noc, t in noc_names:
    c_found = False
    
    f = df_countries['Country or Area'].str.contains(t, case=False, regex=False)
    if any(f):
        # case Country name matches
        region = df_countries.loc[f, 'Region 1'].iloc[0]
        continent = df_countries.loc[f, 'Continent'].iloc[0]
        hdi = df_countries.loc[f, 'HDI'].iloc[0]
        c_found = True
    
        
    else:
        f = df_countries['ISO-alpha3 Code'] == noc
        if any(f):
            # case ISO code matches
            region = df_countries.loc[f, 'Region 1'].iloc[0]
            continent = df_countries.loc[f, 'Continent'].iloc[0]
            hdi = df_countries.loc[f, 'HDI'].iloc[0]
            c_found = True
            
        elif t in countries_converters.keys():
            # case Country name in converter
            region = countries_converters[t][1]
            continent = countries_converters[t][0]
            if t == 'Individual Olympic Athletes':
                # case IOA
                hdi = 0.5
            else:
                hdi = hdi_table.loc[region, 'HDI']
                
            c_found = True
    
    # add information to DataFrame
    if c_found:
        f = df_all_games['NOC'] == noc
        df_all_games.loc[f, 'Region'] = region
        df_all_games.loc[f, 'Continent'] = continent
        df_all_games.loc[f, 'HDI'] = hdi
                
    else:
        if t not in team_not_found:            
            team_not_found.append(t)
            
print('Teams not found [{}]: {}'.format(len(team_not_found), team_not_found))

Teams not found [0]: []


In [20]:
# replace nans in continent as IOC
df_all_games.loc[df_all_games['Continent'] == '', 'Continent'] = 'Individual Olympic Athletes'

## 4.- Save dataset to csv

In [21]:
df_all_games

,Season,Year,City,NOC,F,M,Total Athletes,Gender equity,Team,last_games,Region,Continent,HDI
0,Summer,1896,Athina,AUS,0,5,5,0.000000,Australia,False,Australia and New Zealand,Oceania,0.946
1,Summer,1896,Athina,AUT,0,8,8,0.000000,Austria,False,Western Europe,Europe,0.926
2,Summer,1896,Athina,DEN,0,15,15,0.000000,Denmark,False,Northern Europe,Europe,0.952
3,Summer,1896,Athina,FRA,0,26,26,0.000000,France,False,Western Europe,Europe,0.910
4,Summer,1896,Athina,GBR,0,25,25,0.000000,Great Britain,False,Northern Europe,Europe,0.940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4217,Winter,2022,Beijing,TTO,0,2,2,0.000000,Trinidad and Tobago,True,Caribbean,North America,0.814
4218,Winter,2022,Beijing,TUR,7,7,14,0.500000,Turkey,True,Western Asia,Asia,0.855
4219,Winter,2022,Beijing,UKR,44,46,90,0.488889,Ukraine,True,Eastern Europe,Europe,0.734
4220,Winter,2022,Beijing,USA,174,187,361,0.481994,United States,True,Micronesia,Oceania,0.927


In [22]:
df_all_games.to_csv(r'..\datasets\all_games_ahtletes.csv', index=False)